# Check Config and install right versions of required librairies

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
# AssertionError: MMCV==1.2.5 is used but incompatible. Please install mmcv>=1.2.6, <=1.3.
!pip install mmcv-full==1.2.6 -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.3MB 15kB/s 
     |████████████████████████████████| 12.7MB 197kB/s 
     |████████████████████████████████| 7.6MB 9.0MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.0+cu101 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
     |████████████████████████████████| 24.1MB 130kB/s 
     |████████████████████████████████| 194kB 7.7MB/s 
Cloning into 'mmaction2'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (44/44), done.
remote: 

In [3]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0+cu101 True
0.12.0
10.1
GCC 7.3


# Test MMaction2

In [4]:
# libraries
import os.path as osp
import mmcv

from mmcv import Config
from mmcv.runner import set_random_seed
from mmcv.parallel import MMDataParallel
from mmaction.datasets import build_dataset, build_dataloader
from mmaction.models import build_model
from mmaction.apis import train_model, single_gpu_test, inference_recognizer, init_recognizer


In [5]:
%cd mmaction2/
#prepare checkpoints folder
!rm -rf checkpoints
!mkdir checkpoints

[Errno 2] No such file or directory: 'mmaction2/'
/content/mmaction2


In [22]:
import datetime
#########
# download and return paths for model
#########
# https://github.com/open-mmlab/mmaction2 > scroll model zoo
# slowfast -> click > https://github.com/open-mmlab/mmaction2/blob/master/configs/recognition/slowfast/README.md
# get config AND horizontal scroll to find link to checkpoint file !!!

def get_config_and_chechpoint_model(model_name):
  if model_name == 'slowfast':
    !wget -c https://download.openmmlab.com/mmaction/recognition/slowfast/slowfast_r50_video_4x16x1_256e_kinetics400_rgb/slowfast_r50_video_4x16x1_256e_kinetics400_rgb_20200826-f85b90c5.pth \
          -O checkpoints/slowfast_r50_video_4x16x1_256e_kinetics400_rgb_20200826-f85b90c5.pth
    return 'configs/recognition/slowfast/slowfast_r50_video_4x16x1_256e_kinetics400_rgb.py', 'checkpoints/slowfast_r50_video_4x16x1_256e_kinetics400_rgb_20200826-f85b90c5.pth'
  
  if model_name == 'x3d':
    !wget -c https://download.openmmlab.com/mmaction/recognition/x3d/facebook/x3d_m_facebook_16x5x1_kinetics400_rgb_20201027-3f42382a.pth \
          -O checkpoints/x3d_m_facebook_16x5x1_kinetics400_rgb_20201027-3f42382a.pth
    return 'configs/recognition/x3d/x3d_m_16x5x1_facebook_kinetics400_rgb.py', 'checkpoints/x3d_m_facebook_16x5x1_kinetics400_rgb_20201027-3f42382a.pth'
  
  if model_name == 'tpn':
    !wget -c https://download.openmmlab.com/mmaction/recognition/tpn/tpn_imagenet_pretrained_slowonly_r50_8x8x1_150e_kinetics_rgb/tpn_imagenet_pretrained_slowonly_r50_8x8x1_150e_kinetics_rgb_20200923-52629684.pth \
          -O checkpoints/tpn_imagenet_pretrained_slowonly_r50_8x8x1_150e_kinetics_rgb_20200923-52629684.pth
    return 'configs/recognition/tpn/tpn_imagenet_pretrained_slowonly_r50_8x8x1_150e_kinetics_rgb.py', 'checkpoints/tpn_imagenet_pretrained_slowonly_r50_8x8x1_150e_kinetics_rgb_20200923-52629684.pth'
  
  if model_name == 'tsn':
    !wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
          -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
    return 'configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py', 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

  if model_name == 'tsm':
    !wget -c https://download.openmmlab.com/mmaction/recognition/tsm/tsm_r50_video_1x1x8_100e_kinetics400_rgb/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth \
          -O checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth
    return 'configs/recognition/tsm/tsm_r50_video_1x1x8_50e_kinetics400_rgb.py', 'checkpoints/tsm_r50_video_1x1x8_100e_kinetics400_rgb_20200702-a77f4328.pth'

  if model_name == 'i3d':
    !wget -c https://download.openmmlab.com/mmaction/recognition/i3d/i3d_r50_video_32x2x1_100e_kinetics400_rgb/i3d_r50_video_32x2x1_100e_kinetics400_rgb_20200826-e31c6f52.pth \
          -O checkpoints/i3d_r50_video_32x2x1_100e_kinetics400_rgb_20200826-e31c6f52.pth
    return 'configs/recognition/i3d/i3d_r50_video_32x2x1_100e_kinetics400_rgb.py', 'checkpoints/i3d_r50_video_32x2x1_100e_kinetics400_rgb_20200826-e31c6f52.pth'


#########
# Ensure that config is ok
#########
def test_model(model_name):
  config, checkpoint = get_config_and_chechpoint_model(model_name)
  # Initialize the recognizer
  model = init_recognizer(config, checkpoint, device='cuda:0')

  # Use the recognizer to do inference
  video = 'demo/demo.mp4'
  label = 'demo/label_map_k400.txt'
  start_time = datetime.datetime.now()
  results = inference_recognizer(model, video, label)
  end_time = datetime.datetime.now()
  time_diff = (end_time - start_time)
  execution_time = time_diff.total_seconds()
  
  print("---------------------------------------")
  print(f"{model_name} : execution duration : {execution_time} seconds")

  # Let's show the results
  for result in results:
      print(f'{result[0]}: ', "{:.5f}".format(result[1]))

  print("\n\n")

In [23]:
# test if model
test_model("slowfast")
test_model("tsn")
test_model("i3d")
test_model("tsm")

--2021-03-28 10:49:01--  https://download.openmmlab.com/mmaction/recognition/slowfast/slowfast_r50_video_4x16x1_256e_kinetics400_rgb/slowfast_r50_video_4x16x1_256e_kinetics400_rgb_20200826-f85b90c5.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


---------------------------------------
slowfast : execution duration : 2.64648 seconds
arm wrestling:  0.99932
playing chess:  0.00007
massaging feet:  0.00003
tossing coin:  0.00002
rock scissors paper:  0.00002



--2021-03-28 10:49:05--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.

---------------------------------------
tsn : execution duration : 0.498474 seconds
arm wrestling:  25.66813
rock scissors paper:  9.76665
shaking hands:  8.80924
clapping:  8.33314
stretching leg:  7.59436



--2021-03-28 10:49:07--  https://download.openmmlab.com/mmaction/recognition/i3d/i3d_r50_video_32x2x1_100e_kinetics400_rgb/i3d_r5

# Create custom dataset

In [8]:
# reduce scope of video to use in custom dataset
def dataset_reduce_scope(folder_path, source_filename, target_filename, labels, limit):
  counts = dict((val, 0) for val in labels)
  line_index = 0
  fileTarget = open(osp.join(folder_path, target_filename), 'w')
  fileSource = open(osp.join(folder_path, source_filename), 'r')
  while True:
    line_index = line_index + 1
    line = fileSource.readline()
    if not line:
        break

    is_supported = False
    for label in labels:
      if line.startswith( label ):
        counts[label] = counts[label] + 1
        if counts[label] <= limit:
          is_supported = True

    if is_supported or line_index == 1:
      fileTarget.write(line)

  fileSource.close()
  fileTarget.close()

## kinetics 700

In [9]:
!rm -rf /content/work_dir_kinetics700

In [10]:
# download annotations from dataset provider : here Kinetics 700
# reference : https://github.com/open-mmlab/mmaction2/tree/master/tools/data/kinetics
# before to run : update /content/mmaction2/tools/data/kinetics/download.py set num-jobs to 1 to avoid error on video cut
%cd /content/mmaction2/tools/data/kinetics
!rm -rf /content/mmaction2/data/kinetics700

!bash download_annotations.sh kinetics700

!mv /content/mmaction2/data/kinetics700/annotations/kinetics_train.csv /content/mmaction2/data/kinetics700/annotations/kinetics_train_full.csv
!mv /content/mmaction2/data/kinetics700/annotations/kinetics_val.csv /content/mmaction2/data/kinetics700/annotations/kinetics_val_full.csv
!mv /content/mmaction2/data/kinetics700/annotations/kinetics_test.csv /content/mmaction2/data/kinetics700/annotations/kinetics_test_full.csv

supported_labels = ['falling off chair', 'wrestling', 'yawning']
dataset_reduce_scope('/content/mmaction2/data/kinetics700/annotations/', 'kinetics_train_full.csv', 'kinetics_train.csv', supported_labels, 50)
dataset_reduce_scope('/content/mmaction2/data/kinetics700/annotations/', 'kinetics_val_full.csv', 'kinetics_val.csv', supported_labels, 5)
dataset_reduce_scope('/content/mmaction2/data/kinetics700/annotations/', 'kinetics_test_full.csv', 'kinetics_test.csv', [], 1) # just empty file to avoid crash

# download videos
!bash download_videos.sh kinetics700

# clean up
!bash rename_classnames.sh kinetics700

# resize -> not working !
#!python /content/mmaction2/tools/data/resize_video.py /content/mmaction2/data/kinetics700/videos_train/ /content/mmaction2/data/kinetics700/videos_train_256p_dense_cache --dense --level 2

# prepare files
!bash generate_videos_filelist.sh kinetics700


/content/mmaction2/tools/data/kinetics
We are processing kinetics700
../../../data/kinetics700/annotations does not exist. Creating
--2021-03-28 05:24:22--  https://storage.googleapis.com/deepmind-media/Datasets/kinetics700.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 172.217.2.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24740173 (24M) [application/octet-stream]
Saving to: ‘kinetics700.tar.gz’

kinetics700.tar.gz  100%[===================>]  23.59M  60.6MB/s    in 0.4s    

2021-03-28 05:24:22 (60.6 MB/s) - ‘kinetics700.tar.gz’ saved [24740173/24740173]

kinetics700/validate.json
kinetics700/validate.csv
kinetics700/train.json
kinetics700/train.csv
kinetics700/test.json
kinetics700/test.csv
download_videos.sh: line 4: conda: command not found
download_videos.sh: line 5: activate: No such file or directory
     |███

# Training and Testing

In [14]:
dataset_name = 'kinetics700'
dir_data = '/content/mmaction2/data/'+ dataset_name +'/'
labels = [(l.replace(' ', '_')) for l in supported_labels]

%cd /content/mmaction2/

/content/mmaction2


In [15]:
def create_config_from(config_path, checkpoint_path):
  cfg = Config.fromfile(config_path)

  # Modify the paths
  cfg.dataset_type = 'VideoDataset'
  cfg.data_root = dir_data + 'videos_train'
  cfg.data_root_val = dir_data + 'videos_val'
  cfg.ann_file_train = dir_data + dataset_name + '_train_list_videos.txt'
  cfg.ann_file_val = dir_data + dataset_name  +'_val_list_videos.txt'
  cfg.ann_file_test = dir_data + dataset_name + '_val_list_videos.txt'

  cfg.data.test.type = 'VideoDataset'
  cfg.data.test.ann_file = cfg.ann_file_test
  cfg.data.test.data_prefix = cfg.data_root_val

  cfg.data.val.type = 'VideoDataset'
  cfg.data.val.ann_file = cfg.ann_file_val
  cfg.data.val.data_prefix = cfg.data_root_val

  cfg.data.train.type = 'VideoDataset'
  cfg.data.train.ann_file = cfg.ann_file_train
  cfg.data.train.data_prefix = cfg.data_root

  cfg.setdefault('omnisource', False) # Whether it is omnisource training
  cfg.model.cls_head.num_classes = len(labels)
  cfg.load_from = checkpoint_path # Start the training with a pre-trained checkpoint, or None
  cfg.work_dir = '/content/work_dir_'+ dataset_name
  
  # We use 1 GPU, instead of 8 by default
  # We adjust the learning rate according to the Linear Scaling Rule
  cfg.gpu_ids = range(1)
  cfg.optimizer.lr /= 8

  # Set a new batch size of 4
  old_batch_size = cfg.data.videos_per_gpu
  cfg.data.videos_per_gpu = 4
  # We adjust the learning rate to the new batch size
  cfg.optimizer.lr *= cfg.data.videos_per_gpu / old_batch_size
  cfg.total_epochs = 10 #30

  # We can set the checkpoint saving interval to reduce the storage cost
  cfg.checkpoint_config.interval = 10
  # We can set the log print interval to reduce the the times of printing log
  cfg.log_config.interval = 5

  # Set seed thus the results are more reproducible
  cfg.seed = 0
  set_random_seed(0, deterministic=False)
  cfg.gpu_ids = range(1)
  
  return cfg 

def train(cfg):
  datasets = [build_dataset(cfg.data.train)]

  # https://github.com/open-mmlab/mmaction2/pull/629
  # [Refactor] Config Refactor (train_cfg & test_cfg)
  model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
  mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
  train_model(model, datasets, cfg, distributed=False, validate=True)


def test(cfg):
    model = init_recognizer(cfg, cfg.load_from, device='cuda:0')
    # Build a test dataloader
    dataset = build_dataset(cfg.data.test, dict(test_mode=True))
    data_loader = build_dataloader(
            dataset,
            videos_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)
    model = MMDataParallel(model, device_ids=[0])
    outputs = single_gpu_test(model, data_loader)

    eval_config = cfg.evaluation
    if 'interval' in eval_config:
        eval_config.pop('interval')
    eval_res = dataset.evaluate(outputs, **eval_config)


def train_and_test_model(model_name):
  #get new config for model
  config, checkpoint = get_config_and_chechpoint_model(model_name)

  cfg = create_config_from(config, checkpoint)
  # print(f'Config:\n{cfg.pretty_text}')

  train(cfg)

  cfg.load_from = cfg.work_dir +'/'+ f'epoch_{cfg.total_epochs}.pth'

  test(cfg)

  cfg.dump(cfg.work_dir + '/config.py')


In [16]:
# if failed checked that videos was correct : -> before to run : update /content/mmaction2/tools/data/kinetics/download.py set num-jobs to 1 to avoid error on video cut
train_and_test_model("slowfast")
!mv /content/work_dir_kinetics700 /content/work_dir_kinetics700_slowfast

--2021-03-28 06:57:53--  https://download.openmmlab.com/mmaction/recognition/slowfast/slowfast_r50_video_4x16x1_256e_kinetics400_rgb/slowfast_r50_video_4x16x1_256e_kinetics400_rgb_20200826-f85b90c5.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



2021-03-28 06:57:56,116 - mmaction - INFO - load checkpoint from checkpoints/slowfast_r50_video_4x16x1_256e_kinetics400_rgb_20200826-f85b90c5.pth
2021-03-28 06:57:56,283 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2304]) from checkpoint, the shape in current model is torch.Size([3, 2304]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([3]).
2021-03-28 06:57:56,292 - mmaction - INFO - Start running, host: root@275c845ecfbf, work_dir: /content/work_dir_kinetics700
2021-03-28 06:57:56,293 - mmaction - INFO - workflow: [('train', 1)], max: 10 epochs
/content/mmaction2/mmaction/core/evaluation/eval_hooks.py:131: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
/usr/local/lib/python3.7/dist-packages/torch/nn/fun

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 14/14, 4.1 task/s, elapsed: 3s, ETA:     0s

2021-03-28 07:02:01,246 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-28 07:02:01,248 - mmaction - INFO - 
top1_acc	0.7857
top5_acc	1.0000
2021-03-28 07:02:01,249 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-28 07:02:01,257 - mmaction - INFO - 
mean_acc	0.7667
2021-03-28 07:02:02,225 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2021-03-28 07:02:02,227 - mmaction - INFO - Best top1_acc is 0.7857 at 5 epoch.
2021-03-28 07:02:02,231 - mmaction - INFO - Epoch(val) [5][37]	top1_acc: 0.7857, top5_acc: 1.0000, mean_class_accuracy: 0.7667
2021-03-28 07:02:13,555 - mmaction - INFO - Epoch [6][5/37]	lr: 7.350e-04, eta: 0:03:50, time: 2.263, data_time: 1.130, memory: 4991, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.3265, loss: 0.3265, grad_norm: 6.9516
2021-03-28 07:02:19,988 - mmaction - INFO - Epoch [6][10/37]	lr: 7.462e-04, eta: 0:03:44, time: 1.287, data_time: 0.213, memory: 4991, top1_acc: 0.9500, top5_acc: 1.0000, loss_cls

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 14/14, 4.0 task/s, elapsed: 4s, ETA:     0s

2021-03-28 07:06:07,386 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-28 07:06:07,388 - mmaction - INFO - 
top1_acc	0.7857
top5_acc	1.0000
2021-03-28 07:06:07,390 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-28 07:06:07,394 - mmaction - INFO - 
mean_acc	0.7667
2021-03-28 07:06:07,397 - mmaction - INFO - Epoch(val) [10][37]	top1_acc: 0.7857, top5_acc: 1.0000, mean_class_accuracy: 0.7667


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 14/14, 0.4 task/s, elapsed: 37s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.6429
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	0.6167


In [ ]:
# train other models
train_and_test_model("tsn")
!mv /content/work_dir_kinetics700 /content/work_dir_kinetics700_tsn
train_and_test_model("i3d")
!mv /content/work_dir_kinetics700 /content/work_dir_kinetics700_i3d
train_and_test_model("tsm")
!mv /content/work_dir_kinetics700 /content/work_dir_kinetics700_tsm

In [ ]:
!mkdir my

def run_on_my_video():
  # Initialize the recognizer
  model = init_recognizer('/content/work_dir_kinetics700/config.py', '/content/work_dir_kinetics700/best_top1_acc_epoch_5.pth', device='cuda:0')

  # Use the recognizer to do inference
  video = 'my/Chute.mp4'
  label = 'my/label.txt'
  results = inference_recognizer(model, video, label)

  # Let's show the results
  for result in results:
      print(f'{result[0]}: ', result[1])

run_on_my_video()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


falling off chair:  0.9314812
yawning:  0.03575415
wrestling:  0.03276478
